In [1]:
from datasets import load_dataset, Dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer

In [2]:
dataset = load_dataset('iamjoon/klue-mrc-ko-rag-dataset', split='train')
print(set(dataset['type']))
dataset = dataset.class_encode_column('type')

{'mrc_question_with_1_to_4_negative', 'mrc_question', 'synthetic_question', 'no_answer', 'paraphrased_question'}


In [3]:
type_name_dict = {1:'mrc_question_with_1_to_4_negative', 3:'paraphrased_question', 2:'no_answer', 4:'synthetic_question', 0:'mrc_question'}

In [4]:
system_message = '''
당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문에 대한 내용이 없습니다." 라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 예를 들어 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]라고 기재하십시오.
6. 최대한 다수의 문서를 인용하여 답변하십시오.

검색 결과:
----------
{search_result}'''

In [5]:
print('원본 데이터의 type 분포:')
for type_name in set(dataset['type']):
    print(f'{type_name_dict[type_name]}: {dataset['type'].count(type_name)}')

원본 데이터의 type 분포:
mrc_question: 491
mrc_question_with_1_to_4_negative: 296
no_answer: 404
paraphrased_question: 196
synthetic_question: 497


In [6]:
split_dataset = dataset.train_test_split(test_size=0.5, stratify_by_column='type')
train_dataset_format = split_dataset['train']
test_dataset_format = split_dataset['test']

In [7]:
def format_data(sample):
    search_result = '\n------\n'.join(f'문서{idx+1}: {result}' for idx, result in enumerate(sample['search_result']))

    return {
        'messages':[{'role':'system', 'content':system_message.format(search_result=search_result)},
                    {'role':'user', 'content':sample['question']},
                    {'role':'assistant', 'content':sample['answer']}]
    }

In [8]:
train_dataset = [format_data(train_data) for train_data in train_dataset_format]
test_dataset = [format_data(test_data) for test_data in test_dataset_format]

In [9]:
print(f'\n전체 데이터 분할 결과: Train {len(train_dataset)}개, Test {len(test_dataset)}개')
print('--'*20)
print('\n학습 데이터의 type 분포:')
for type_name in set(dataset['type']):
    print(f'{type_name_dict[type_name]}: {train_dataset_format['type'].count(type_name)}')
print('\n테스트 데이터의 type 분포:')
for type_name in set(dataset['type']):
    print(f'{type_name_dict[type_name]}: {test_dataset_format['type'].count(type_name)}')


전체 데이터 분할 결과: Train 942개, Test 942개
----------------------------------------

학습 데이터의 type 분포:
mrc_question: 245
mrc_question_with_1_to_4_negative: 148
no_answer: 202
paraphrased_question: 98
synthetic_question: 249

테스트 데이터의 type 분포:
mrc_question: 246
mrc_question_with_1_to_4_negative: 148
no_answer: 202
paraphrased_question: 98
synthetic_question: 248
